In [1]:
!python -m pip install boto3

In [2]:
import pandas as pd
import boto3
import json

# Load DWH Params from a file

In [3]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('../airflow_redshift.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("CLUSTER","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("CLUSTER","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("CLUSTER","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("CLUSTER","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("CLUSTER","DB_NAME")
DWH_DB_USER            = config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DB_PORT")

DWH_IAM_ROLE_NAME      = config.get("CLUSTER", "DWH_IAM_ROLE_NAME")


(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dags-cluster
4,DWH_DB,dags-testing
5,DWH_DB_USER,dags-user
6,DWH_DB_PASSWORD,Dagsuser1
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,sparkifyDataLakeRole


## Create clients for EC2, IAM, and Redshift

In [4]:
import boto3

ec2 = boto3.resource('ec2',
                       region_name='us-east-1',
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)

iam = boto3.client('iam',
                       region_name='us-east-1',
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)

redshift = boto3.client('redshift',
                       region_name='us-east-1',
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)

## Create IAM Role

In [11]:
try:
    dwhRole = iam.create_role(Path='/', RoleName=DWH_IAM_ROLE_NAME,
                             Description='Allows Redshift clusters to call AWS services',
                             AssumeRolePolicyDocument=json.dumps({'Statement':
                                                                 [{'Action':'sts:AssumeRole',
                                                                 'Effect':'Allow',
                                                                 'Principal':{'Service':'redshift.amazonaws.com'}}],
                             'Version': '2012-10-17'}))
    

except Exception as e:
    print(e)


## Attach role policy

In [12]:
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

200

## Check role

In [5]:
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

arn:aws:iam::244864396415:role/sparkifyDataLakeRole


##  Create Cluster
- Create a RedShift Cluster
- For complete arguments to `create_cluster`, see [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster)

In [14]:
try:
    response = redshift.create_cluster(         
        #hardware
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),
        #identifiers & credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        #role (to allow s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

## *Describe* the cluster to see its status


In [6]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

C:\Users\mitch\Anaconda3\envs\data_eng\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Key,Value
0,ClusterIdentifier,dags-cluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dags-user
4,DBName,dags-testing
5,Endpoint,"{'Address': 'dags-cluster.c2yxu1dbnzfx.us-east-1.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-666bc91b
7,NumberOfNodes,4


In [7]:
import time
while True:
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
    if myClusterProps['ClusterStatus'] == 'available':
        print('cluster ready')
        break
    else:
        time.sleep(5)

cluster ready


## Check Endpoint

In [8]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  dags-cluster.c2yxu1dbnzfx.us-east-1.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::244864396415:role/sparkifyDataLakeRole


## Open an incoming  TCP port to access the cluster ednpoint

In [9]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-01a0d1082413be67d')


## STEP 4: Make sure you can connect to the clusterConnect to the cluster

In [17]:
!pip install ipython-sql

  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13704 sha256=06cff7eed0ffdac74eede0b2cc17951e1e39b5631cfc1fb9c9a3cff0ed1003a4
  Stored in directory: c:\users\mitch\appdata\local\pip\cache\wheels\90\15\c3\5f28b42ae9c81638570b8b7ed654e0f98c5fdc08875869511b
Successfully built prettytable


In [18]:
%load_ext sql

In [22]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
#print(conn_string)
%sql $conn_string

In [23]:
#print(conn_string)

In [20]:
%sql $conn_string

In [13]:
!pip install psycopg2

In [14]:
import psycopg2

config.read('dwh.cfg')
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(DWH_ENDPOINT, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT))

In [15]:
myClusterProps

{'ClusterIdentifier': 'dags-cluster',
 'NodeType': 'dc2.large',
 'ClusterStatus': 'available',
 'ClusterAvailabilityStatus': 'Available',
 'MasterUsername': 'dags-user',
 'DBName': 'dags-testing',
 'Endpoint': {'Address': 'dags-cluster.c2yxu1dbnzfx.us-east-1.redshift.amazonaws.com',
  'Port': 5439},
 'ClusterCreateTime': datetime.datetime(2021, 3, 21, 14, 47, 23, 67000, tzinfo=tzutc()),
 'AutomatedSnapshotRetentionPeriod': 1,
 'ManualSnapshotRetentionPeriod': -1,
 'ClusterSecurityGroups': [],
 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-6f969d64',
   'Status': 'active'}],
 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
   'ParameterApplyStatus': 'in-sync'}],
 'ClusterSubnetGroupName': 'default',
 'VpcId': 'vpc-666bc91b',
 'AvailabilityZone': 'us-east-1c',
 'PreferredMaintenanceWindow': 'sun:10:30-sun:11:00',
 'PendingModifiedValues': {},
 'ClusterVersion': '1.0',
 'AllowVersionUpgrade': True,
 'NumberOfNodes': 4,
 'PubliclyAccessible': True,
 'Encrypted'

In [16]:
prettyRedshiftProps(myClusterProps)

C:\Users\mitch\Anaconda3\envs\data_eng\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Key,Value
0,ClusterIdentifier,dags-cluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dags-user
4,DBName,dags-testing
5,Endpoint,"{'Address': 'dags-cluster.c2yxu1dbnzfx.us-east-1.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-666bc91b
7,NumberOfNodes,4
